<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Acknowledgements" data-toc-modified-id="Acknowledgements-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Acknowledgements</a></span></li><li><span><a href="#Prepare-data-and-model" data-toc-modified-id="Prepare-data-and-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare data and model</a></span></li><li><span><a href="#Make-feature-matrix-(word2vec,-votes,-stars)" data-toc-modified-id="Make-feature-matrix-(word2vec,-votes,-stars)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Make feature matrix (word2vec, votes, stars)</a></span></li><li><span><a href="#Create-Label-y-(Business-categories)" data-toc-modified-id="Create-Label-y-(Business-categories)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Label y (Business categories)</a></span></li><li><span><a href="#Join-x,y-(feature-matrix,-category)-using-business_id" data-toc-modified-id="Join-x,y-(feature-matrix,-category)-using-business_id-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Join x,y (feature matrix, category) using business_id</a></span></li><li><span><a href="#Category-Prediction" data-toc-modified-id="Category-Prediction-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Category Prediction</a></span><ul class="toc-item"><li><span><a href="#Recall-(and-other-classification-metrics)" data-toc-modified-id="Recall-(and-other-classification-metrics)-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Recall (and other classification metrics)</a></span></li><li><span><a href="#Top-RECOMMENDATIONS" data-toc-modified-id="Top-RECOMMENDATIONS-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Top RECOMMENDATIONS</a></span></li></ul></li><li><span><a href="#Cluster-with-metadata-(useful,-cool,-funny,-stars)" data-toc-modified-id="Cluster-with-metadata-(useful,-cool,-funny,-stars)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Cluster with metadata (useful, cool, funny, stars)</a></span></li></ul></div>

# Acknowledgements
Thanks to the tutorial: https://www.kaggle.com/c/word2vec-nlp-tutorial/overview/part-3-more-fun-with-word-vectors

# Prepare data and model

In [76]:
%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows=999
import numpy as np
import matplotlib.pyplot as plt

import re

import nltk
import nltk.data
nltk.download('stopwords')
from nltk.corpus import stopwords # Import the stop word list



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daviderickson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def load_reviews(size='small'): 
    if size == 'small':
        filename = r'../../data/small-review.json'
    elif size == 'intermediate':
        filename = r'../../data/intermediate-review.json'
    elif size == 'full':
        filename = r'../../data/review.json'
    new_list = []
    for line in open(filename):
       new_list.append(json.loads(line))
    return pd.DataFrame.from_records(new_list)

dfreviews = load_reviews(size='intermediate')

In [3]:
dfreviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


In [4]:
dfreviews.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')

In [5]:
dfreviews['text'][0]

'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.'

In [6]:
# For simplicity, drop anything that isn't a letter
# Numbers and symbols may have interesting meaning and could be explore later

def lettersOnly(string):
    return re.sub("[^a-zA-Z]", " ", string) 

dfreviews['text'] = dfreviews['text'].apply(lettersOnly)


In [7]:
dfreviews['text'][0]

'Total bill for this horrible service  Over   Gs  These crooks actually had the nerve to charge us     for   pills  I checked online the pills can be had for    cents EACH  Avoid Hospital ERs at all costs '

In [8]:
def review_to_wordlist(string, remove_stopwords=False):
    string = re.sub("[^a-zA-Z]", " ", string) # keep only letters. more complex model possible later
    words =  string.lower().split() # make everything lowercase. split into words
    if remove_stopwords:
        stops = set(stopwords.words('english')) # create a fast lookup for stopwords
        words = [w for w in words if not w in stops] # remove stopwords
    return( words) # return a list of words
    
# dfreviews['text'] = dfreviews['text'].apply(review_to_words) # apply to reviews in dataframe


In [9]:
# Word2Vec expects single sentences, each one as a list of words

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [10]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences")
for review in dfreviews["text"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences


In [11]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2020-01-21 21:41:36,516 : INFO : 'pattern' package not found; tag filters are not available for English
2020-01-21 21:41:36,528 : INFO : collecting all words and their counts
2020-01-21 21:41:36,528 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2020-01-21 21:41:36,729 : INFO : PROGRESS: at sentence #10000, processed 1088334 words, keeping 25539 word types
2020-01-21 21:41:36,911 : INFO : PROGRESS: at sentence #20000, processed 2172597 words, keeping 35463 word types
2020-01-21 21:41:37,086 : INFO : PROGRESS: at sentence #30000, processed 3251616 words, keeping 42649 word types
2020-01-21 21:41:37,292 : INFO : PROGRESS: at sentence #40000, processed 4373996 words, keeping 48893 word types
2020-01-21 21:41:37,474 : INFO : PROGRESS: at sentence #50000, processed 5471587 words, keeping 53964 word types
2020-01-21 21:41:37,649 : INFO : PROGRESS: at sentence #60000, processed 6570064 words, keeping 58362 word types
2020-01-21 21:41:37,832 : INFO : PROGRESS: at sentence #70000, processed 7667364 words, keeping 62704 word types
2020-01-21 21:41:38,004 : INFO : PROGRESS: at sentence #80000, processed 8768955 words, keeping 66443 word types
2020-01-21 21:41:38,176 : INFO : PROGRESS: at sentence #90000, processed 9872097 words, keeping 

2020-01-21 21:42:20,436 : INFO : EPOCH 5 - PROGRESS: at 20.53% examples, 789415 words/s, in_qsize 7, out_qsize 0
2020-01-21 21:42:21,448 : INFO : EPOCH 5 - PROGRESS: at 30.26% examples, 772301 words/s, in_qsize 7, out_qsize 0
2020-01-21 21:42:22,451 : INFO : EPOCH 5 - PROGRESS: at 40.21% examples, 777214 words/s, in_qsize 7, out_qsize 0
2020-01-21 21:42:23,452 : INFO : EPOCH 5 - PROGRESS: at 50.41% examples, 780440 words/s, in_qsize 7, out_qsize 0
2020-01-21 21:42:24,458 : INFO : EPOCH 5 - PROGRESS: at 60.76% examples, 784652 words/s, in_qsize 7, out_qsize 0
2020-01-21 21:42:25,474 : INFO : EPOCH 5 - PROGRESS: at 71.12% examples, 786514 words/s, in_qsize 7, out_qsize 0
2020-01-21 21:42:26,474 : INFO : EPOCH 5 - PROGRESS: at 79.34% examples, 768400 words/s, in_qsize 7, out_qsize 0
2020-01-21 21:42:27,475 : INFO : EPOCH 5 - PROGRESS: at 89.58% examples, 771905 words/s, in_qsize 7, out_qsize 0
2020-01-21 21:42:28,461 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-

In [12]:
model.most_similar('pizza')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('crust', 0.7070269584655762),
 ('pizzas', 0.6834176182746887),
 ('pepperoni', 0.6815404891967773),
 ('calzone', 0.6130505800247192),
 ('margherita', 0.6126407384872437),
 ('dough', 0.5469943284988403),
 ('mozzarella', 0.5322738885879517),
 ('slice', 0.5247739553451538),
 ('lasagna', 0.5243027210235596),
 ('meatball', 0.5210219621658325)]

In [13]:
model.most_similar('service')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('waitstaff', 0.5219206809997559),
 ('staff', 0.46909892559051514),
 ('servers', 0.43119579553604126),
 ('hospitality', 0.41726261377334595),
 ('communication', 0.40516912937164307),
 ('execution', 0.4035489559173584),
 ('ambience', 0.401478111743927),
 ('consistently', 0.3970610499382019),
 ('value', 0.39341211318969727),
 ('bartenders', 0.3915520906448364)]

In [14]:
model.most_similar('bad')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.6198078393936157),
 ('horrible', 0.5894107222557068),
 ('good', 0.5501216650009155),
 ('poor', 0.544106125831604),
 ('awful', 0.5215410590171814),
 ('alright', 0.4838787317276001),
 ('disappointing', 0.4818119704723358),
 ('greatest', 0.4734565019607544),
 ('crappy', 0.45638489723205566),
 ('pathetic', 0.4444549083709717)]

In [15]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # WV.Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = int(0.)
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [16]:
# ****************************************************************
# Calculate average feature vectors
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_reviews = []
for review in dfreviews["text"]:
    clean_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

reviewDataVecs = getAvgFeatureVecs( clean_reviews, model, num_features )

Review 0 of 100000


/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 100000
Review 2000 of 100000
Review 3000 of 100000
Review 4000 of 100000
Review 5000 of 100000
Review 6000 of 100000
Review 7000 of 100000
Review 8000 of 100000


/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


Review 9000 of 100000
Review 10000 of 100000
Review 11000 of 100000
Review 12000 of 100000
Review 13000 of 100000
Review 14000 of 100000
Review 15000 of 100000
Review 16000 of 100000
Review 17000 of 100000
Review 18000 of 100000
Review 19000 of 100000
Review 20000 of 100000
Review 21000 of 100000
Review 22000 of 100000
Review 23000 of 100000
Review 24000 of 100000
Review 25000 of 100000
Review 26000 of 100000
Review 27000 of 100000
Review 28000 of 100000
Review 29000 of 100000
Review 30000 of 100000
Review 31000 of 100000
Review 32000 of 100000
Review 33000 of 100000
Review 34000 of 100000
Review 35000 of 100000
Review 36000 of 100000
Review 37000 of 100000
Review 38000 of 100000
Review 39000 of 100000
Review 40000 of 100000
Review 41000 of 100000
Review 42000 of 100000
Review 43000 of 100000
Review 44000 of 100000
Review 45000 of 100000
Review 46000 of 100000
Review 47000 of 100000
Review 48000 of 100000
Review 49000 of 100000
Review 50000 of 100000
Review 51000 of 100000
Review 52000

# Make feature matrix (word2vec, votes, stars)

In [17]:
reviewDataVecs.shape[1]

300

In [18]:
# Add non-text data back to feature matrix
review_features = ['cool', 'funny', 'useful', 'stars' , 'business_id']
all_features_labels = ['w2v{}'.format(idx) for idx in range(reviewDataVecs.shape[1])] + review_features
all_features = np.append(reviewDataVecs, dfreviews[review_features].to_numpy(), 1)


In [19]:
# Create df 
all_features_df = pd.DataFrame(data=all_features, columns=all_features_labels)

# Convert all but business_id to numerical
business_ids = all_features_df['business_id']
all_features_df = all_features_df.iloc[:,:-1].astype('float64')
all_features_df['business_id'] = business_ids
del business_ids

# Group by business_id
all_features_business = all_features_df.groupby(by='business_id').mean()

In [20]:
all_features_business.head()

,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars
business_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--I7YYLada0tSLkORTHb5Q,-0.013726,-0.000967,0.009651,-0.009130,0.016051,0.000565,-0.006745,0.006949,0.021217,0.017539,0.002967,-0.003814,0.016394,0.011443,-0.010743,0.015422,-0.007492,0.006412,-0.018508,-0.003763,-0.010258,-0.034641,0.003971,0.012807,0.020696,0.008306,0.000858,0.016784,-0.017261,-0.019222,0.018138,0.002409,0.030819,0.007352,-0.009905,0.003296,0.009896,0.020073,0.009849,-0.011995,-0.003346,0.018278,0.011761,-0.008173,-0.002904,0.000559,-0.023254,-0.000756,0.014323,0.021623,0.009647,-0.000923,0.018409,0.004140,-0.015980,0.003323,-0.012654,0.005351,-0.010580,0.004594,-0.014951,0.002638,-0.001018,0.019544,0.009782,0.034841,-0.003797,0.001427,-0.016187,-0.008701,-0.004137,-0.003884,0.012173,-0.004549,-0.008592,-0.019691,-0.011440,-0.001867,0.003111,-0.013886,0.008983,0.011444,0.007860,0.005983,0.007405,0.007301,0.015340,-0.007426,0.025241,-0.018652,-0.003055,0.006571,-0.003629,0.005686,0.008752,0.000279,-0.017226,0.014510,-0.003971,-0.003905,0.016058,-0.022591,0.009685,0.020854,-0.006142,-0.005774,0.009259,0.008155,-0.007269,-0.007763,0.006829,-0.021465,-0.007562,0.003837,-0.003232,-0.010939,-0.001754,-0.019062,-0.013329,-0.009903,-0.001265,0.021219,-0.003071,0.002521,-0.009624,-0.006912,-0.008346,0.018096,-0.008363,-0.004684,0.006336,0.012810,-0.000807,0.008201,0.012035,-0.009213,-0.009921,0.007764,0.008899,0.007216,-0.005796,0.006133,0.013801,-0.015935,-0.007177,0.002236,0.003129,-0.012927,-0.006159,0.004225,0.026937,0.001358,0.012673,-0.004315,-0.002035,0.008295,-0.001315,0.005393,0.003733,0.024581,-0.019670,0.008099,0.003728,-0.018815,0.001010,-0.008273,0.000780,0.007821,-0.010501,0.013363,-0.017681,-0.014448,0.004403,0.000807

In [21]:
all_features_business.describe()

,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars
count,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,13942.000000,139

# Create Label y (Business categories)

In [22]:
def load_business_df(): 
    filename = r'../../data/business.json'
    new_list = []
    for line in open(filename):
       new_list.append(json.loads(line))
    return pd.DataFrame.from_records(new_list)

dfbusiness = load_business_df()

In [23]:
dfbusiness.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
3,"15655 W Roosevelt St, Ste 237",None,xvX2CttrVhyG2z1dFg_0xw,"Insurance, Financial Services",Goodyear,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,33.455613,-112.395596,Farmers Insurance - Paul Lorenz,85338,3,5.0,AZ
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC


# Join x,y (feature matrix, category) using business_id

In [24]:
dfbusiness.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')

In [25]:
len(dfbusiness['stars'].unique())

9

In [26]:
# Add business details to features df
keep_cols = ['business_id', 'categories', 'review_count']
all_features_business = all_features_business.merge(dfbusiness[keep_cols], how='left', on='business_id') 

In [27]:
all_features_business.head()

,business_id,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars,categories,review_count
0,--I7YYLada0tSLkORTHb5Q,-0.013726,-0.000967,0.009651,-0.009130,0.016051,0.000565,-0.006745,0.006949,0.021217,0.017539,0.002967,-0.003814,0.016394,0.011443,-0.010743,0.015422,-0.007492,0.006412,-0.018508,-0.003763,-0.010258,-0.034641,0.003971,0.012807,0.020696,0.008306,0.000858,0.016784,-0.017261,-0.019222,0.018138,0.002409,0.030819,0.007352,-0.009905,0.003296,0.009896,0.020073,0.009849,-0.011995,-0.003346,0.018278,0.011761,-0.008173,-0.002904,0.000559,-0.023254,-0.000756,0.014323,0.021623,0.009647,-0.000923,0.018409,0.004140,-0.015980,0.003323,-0.012654,0.005351,-0.010580,0.004594,-0.014951,0.002638,-0.001018,0.019544,0.009782,0.034841,-0.003797,0.001427,-0.016187,-0.008701,-0.004137,-0.003884,0.012173,-0.004549,-0.008592,-0.019691,-0.011440,-0.001867,0.003111,-0.013886,0.008983,0.011444,0.007860,0.005983,0.007405,0.007301,0.015340,-0.007426,0.025241,-0.018652,-0.003055,0.006571,-0.003629,0.005686,0.008752,0.000279,-0.017226,0.014510,-0.003971,-0.003905,0.016058,-0.022591,0.009685,0.020854,-0.006142,-0.005774,0.009259,0.008155,-0.007269,-0.007763,0.006829,-0.021465,-0.007562,0.003837,-0.003232,-0.010939,-0.001754,-0.019062,-0.013329,-0.009903,-0.001265,0.021219,-0.003071,0.002521,-0.009624,-0.006912,-0.008346,0.018096,-0.008363,-0.004684,0.006336,0.012810,-0.000807,0.008201,0.012035,-0.009213,-0.009921,0.007764,0.008899,0.007216,-0.005796,0.006133,0.013801,-0.015935,-0.007177,0.002236,0.003129,-0.012927,-0.006159,0.004225,0.026937,0.001358,0.012673,-0.004315,-0.002035,0.008295,-0.001315,0.005393,0.003733,0.024581,-0.019670,0.008099,0.003728,-0.018815,0.001010,-0.008273,0.000780,0.007821,-0.010501,0.013363,-0.017681,-0.014448,0.004403,0.000807,-0.007189,0.008730,0.000223,0.014109,0.030282,-0.002383,-0.013673,-0.003500,-0.007390,0.013017,-0.007018,-0.018764,0.004093,0.001161,-0.003004,0.000460,-0.011640,-0.014427,0.006012,-0.003372,0.008276,0.000163,0.005292,0.009312,0.005778,0.014780,0.010748,0.006528,-0.011686,-0.0

In [28]:
all_features_business['categories'][0]

'Nightlife, Sports Bars, Restaurants, Bars, American (Traditional)'

In [29]:
all_features_business.head()

,business_id,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars,categories,review_count
0,--I7YYLada0tSLkORTHb5Q,-0.013726,-0.000967,0.009651,-0.009130,0.016051,0.000565,-0.006745,0.006949,0.021217,0.017539,0.002967,-0.003814,0.016394,0.011443,-0.010743,0.015422,-0.007492,0.006412,-0.018508,-0.003763,-0.010258,-0.034641,0.003971,0.012807,0.020696,0.008306,0.000858,0.016784,-0.017261,-0.019222,0.018138,0.002409,0.030819,0.007352,-0.009905,0.003296,0.009896,0.020073,0.009849,-0.011995,-0.003346,0.018278,0.011761,-0.008173,-0.002904,0.000559,-0.023254,-0.000756,0.014323,0.021623,0.009647,-0.000923,0.018409,0.004140,-0.015980,0.003323,-0.012654,0.005351,-0.010580,0.004594,-0.014951,0.002638,-0.001018,0.019544,0.009782,0.034841,-0.003797,0.001427,-0.016187,-0.008701,-0.004137,-0.003884,0.012173,-0.004549,-0.008592,-0.019691,-0.011440,-0.001867,0.003111,-0.013886,0.008983,0.011444,0.007860,0.005983,0.007405,0.007301,0.015340,-0.007426,0.025241,-0.018652,-0.003055,0.006571,-0.003629,0.005686,0.008752,0.000279,-0.017226,0.014510,-0.003971,-0.003905,0.016058,-0.022591,0.009685,0.020854,-0.006142,-0.005774,0.009259,0.008155,-0.007269,-0.007763,0.006829,-0.021465,-0.007562,0.003837,-0.003232,-0.010939,-0.001754,-0.019062,-0.013329,-0.009903,-0.001265,0.021219,-0.003071,0.002521,-0.009624,-0.006912,-0.008346,0.018096,-0.008363,-0.004684,0.006336,0.012810,-0.000807,0.008201,0.012035,-0.009213,-0.009921,0.007764,0.008899,0.007216,-0.005796,0.006133,0.013801,-0.015935,-0.007177,0.002236,0.003129,-0.012927,-0.006159,0.004225,0.026937,0.001358,0.012673,-0.004315,-0.002035,0.008295,-0.001315,0.005393,0.003733,0.024581,-0.019670,0.008099,0.003728,-0.018815,0.001010,-0.008273,0.000780,0.007821,-0.010501,0.013363,-0.017681,-0.014448,0.004403,0.000807,-0.007189,0.008730,0.000223,0.014109,0.030282,-0.002383,-0.013673,-0.003500,-0.007390,0.013017,-0.007018,-0.018764,0.004093,0.001161,-0.003004,0.000460,-0.011640,-0.014427,0.006012,-0.003372,0.008276,0.000163,0.005292,0.009312,0.005778,0.014780,0.010748,0.006528,-0.011686,-0.0

In [30]:
def stringDFColToBinaryCols(df, series_name):
    # Create list of all categories
    all_cats = []
    for string in df[series_name]:
        string = str(string)
        cats = string.strip().replace(' ', '').split(',')
        for cat in cats:
            if cat not in all_cats:
                all_cats.append(cat)
    # Make binary for each cat for each row
    for cat in all_cats:
        df[cat] = df[series_name].str.strip().str.replace(' ', '').str.contains(cat)
        # This technique will have some problems. 'Golf' may appear in non-Golf categories (ie 'Disc Golf')
        # Can be fixed with regular expressions: ',Golf,' OR 'BOF Golf,' OR ',Golf EOF'
    
    return df, all_cats
        
all_features_business, all_cats = stringDFColToBinaryCols(all_features_business, 'categories')

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


In [31]:
print(all_cats)

['Nightlife', 'SportsBars', 'Restaurants', 'Bars', 'American(Traditional)', 'Pizza', 'HairRemoval', 'NailTechnicians', 'Beauty&Spas', 'NailSalons', 'Waxing', 'DaySpas', 'Electronics', 'ProfessionalServices', 'LocalServices', 'ElectronicsRepair', 'Computers', 'Shopping', 'Automotive', 'AutoRepair', 'Chinese', 'EyelashService', 'TobaccoShops', 'VapeShops', 'CarDealers', 'UsedCarDealers', 'Dentists', 'GeneralDentistry', 'CosmeticDentists', 'PediatricDentists', 'Health&Medical', 'Tex-Mex', 'Mexican', 'Arts&Entertainment', 'Festivals', 'Food', 'FoodTrucks', 'FarmersMarket', 'Portuguese', 'Bakeries', 'ChickenShop', 'Barbeque', 'EventPlanning&Services', 'EventPhotography', 'Photographers', 'SessionPhotography', 'SkinCare', 'Antiques', 'IceCream&FrozenYogurt', 'Donuts', 'SpecialtyFood', 'WebDesign', 'GraphicDesign', 'Marketing', 'RecyclingCenter', 'Caterers', 'Southern', 'ComfortFood', 'Breakfast&Brunch', 'French', 'American(New)', 'Burgers', 'Sandwiches', 'Coffee&Tea', 'Brasseries', 'Gyms', '

In [32]:
print(
    len(all_features_business[all_features_business['Golf']==True]), 
    len(all_features_business[all_features_business['DiscGolf']==True]), 
)

61 1


In [33]:
print(all_features_business[all_features_business['DiscGolf']==True]['categories'].values)
print('Should not have a True value for Golf, but does. Problem to deal with in the future.')
print(all_features_business[all_features_business['DiscGolf']==True]['Golf'].values)

['Sporting Goods, Active Life, Bike Rentals, Disc Golf, Shopping']
Should not have a True value for Golf, but does. Problem to deal with in the future.
[True]


In [34]:
all_features_business.head()

,business_id,w2v0,w2v1,w2v2,w2v3,w2v4,w2v5,w2v6,w2v7,w2v8,w2v9,w2v10,w2v11,w2v12,w2v13,w2v14,w2v15,w2v16,w2v17,w2v18,w2v19,w2v20,w2v21,w2v22,w2v23,w2v24,w2v25,w2v26,w2v27,w2v28,w2v29,w2v30,w2v31,w2v32,w2v33,w2v34,w2v35,w2v36,w2v37,w2v38,w2v39,w2v40,w2v41,w2v42,w2v43,w2v44,w2v45,w2v46,w2v47,w2v48,w2v49,w2v50,w2v51,w2v52,w2v53,w2v54,w2v55,w2v56,w2v57,w2v58,w2v59,w2v60,w2v61,w2v62,w2v63,w2v64,w2v65,w2v66,w2v67,w2v68,w2v69,w2v70,w2v71,w2v72,w2v73,w2v74,w2v75,w2v76,w2v77,w2v78,w2v79,w2v80,w2v81,w2v82,w2v83,w2v84,w2v85,w2v86,w2v87,w2v88,w2v89,w2v90,w2v91,w2v92,w2v93,w2v94,w2v95,w2v96,w2v97,w2v98,w2v99,w2v100,w2v101,w2v102,w2v103,w2v104,w2v105,w2v106,w2v107,w2v108,w2v109,w2v110,w2v111,w2v112,w2v113,w2v114,w2v115,w2v116,w2v117,w2v118,w2v119,w2v120,w2v121,w2v122,w2v123,w2v124,w2v125,w2v126,w2v127,w2v128,w2v129,w2v130,w2v131,w2v132,w2v133,w2v134,w2v135,w2v136,w2v137,w2v138,w2v139,w2v140,w2v141,w2v142,w2v143,w2v144,w2v145,w2v146,w2v147,w2v148,w2v149,w2v150,w2v151,w2v152,w2v153,w2v154,w2v155,w2v156,w2v157,w2v158,w2v159,w2v160,w2v161,w2v162,w2v163,w2v164,w2v165,w2v166,w2v167,w2v168,w2v169,w2v170,w2v171,w2v172,w2v173,w2v174,w2v175,w2v176,w2v177,w2v178,w2v179,w2v180,w2v181,w2v182,w2v183,w2v184,w2v185,w2v186,w2v187,w2v188,w2v189,w2v190,w2v191,w2v192,w2v193,w2v194,w2v195,w2v196,w2v197,w2v198,w2v199,w2v200,w2v201,w2v202,w2v203,w2v204,w2v205,w2v206,w2v207,w2v208,w2v209,w2v210,w2v211,w2v212,w2v213,w2v214,w2v215,w2v216,w2v217,w2v218,w2v219,w2v220,w2v221,w2v222,w2v223,w2v224,w2v225,w2v226,w2v227,w2v228,w2v229,w2v230,w2v231,w2v232,w2v233,w2v234,w2v235,w2v236,w2v237,w2v238,w2v239,w2v240,w2v241,w2v242,w2v243,w2v244,w2v245,w2v246,w2v247,w2v248,w2v249,w2v250,w2v251,w2v252,w2v253,w2v254,w2v255,w2v256,w2v257,w2v258,w2v259,w2v260,w2v261,w2v262,w2v263,w2v264,w2v265,w2v266,w2v267,w2v268,w2v269,w2v270,w2v271,w2v272,w2v273,w2v274,w2v275,w2v276,w2v277,w2v278,w2v279,w2v280,w2v281,w2v282,w2v283,w2v284,w2v285,w2v286,w2v287,w2v288,w2v289,w2v290,w2v291,w2v292,w2v293,w2v294,w2v295,w2v296,w2v297,w2v298,w2v299,cool,funny,useful,stars,categories,review_count,Nightlife,SportsBars,Restaurants,Bars,American(Traditional),Pizza,HairRemoval,NailTechnicians,Beauty&Spas,NailSalons,Waxing,DaySpas,Electronics,ProfessionalServices,LocalServices,ElectronicsRepair,Computers,Shopping,Automotive,AutoRepair,Chinese,EyelashService,TobaccoShops,VapeShops,CarDealers,UsedCarDealers,Dentists,GeneralDentistry,CosmeticDentists,PediatricDentists,Health&Medical,Tex-Mex,Mexican,Arts&Entertainment,Festivals,Food,FoodTrucks,FarmersMarket,Portuguese,Bakeries,ChickenShop,Barbeque,EventPlanning&Services,EventPhotography,Photographers,SessionPhotography,SkinCare,Antiques,IceCream&FrozenYogurt,Donuts,SpecialtyFood,WebDesign,GraphicDesign,Marketing,RecyclingCenter,Caterers,Southern,ComfortFood,Breakfast&Brunch,French,American(New),Burgers,Sandwiches,Coffee&Tea,Brasseries,Gyms,ChildCare&DayCare,LeisureCenters,Fitness&Instruction,ActiveLife,HardwareStores,Home&Garden,RealEstate,Condominiums,Hotels,HomeServices,ShoppingCenters,Hotels&Travel,HairSalons,EthnicFood,Turkish,InternationalGrocery,TapasBars,ShippingCenters,PrintingServices,Massage,MassageTherapy,Reflexology,Buffets,Korean,SushiBars,Japanese,Cafes,Soup,Golf,Venues&EventSpaces,AutoDetailing,BodyShops,AutoCustomization,Towing,Trainers,WeightLossCenters,FoodDeliveryServices,FastFood,Delis,Ethiopian,Vegetarian,Painters,DrywallInstallation&Repair,StuccoServices,Orthodontists,Periodontists,OralSurgeons,Piercing,Tattoo,Chiropractors,Optometrists,Italian,Couriers&DeliveryServices,PublicServices&Government,SportingGoods,Fashion,GolfEquipment,Bikes,Ski&SnowboardShops,SportsWear,BikeRepair/Maintenance,Filipino,PetGroomers,Veterinarians,PetSitting,Pets,PetServices,AutoGlassServices,RealEstateServices,RealEstateAgents,Pakistani,Indian,CardioClasses,DanceStudios,ChickenWings,Cosmetics&BeautySupply,Desserts,Sewing&Alterations,Arts&Crafts,Wheel&RimRepair,Tires,AutoParts&Supplies,Colonics,Saunas,Doctors,MedicalSpas,Naturopathic/Holistic,MeditationCenters,Reiki,Spi

In [35]:
# Clean

# Remove rows with NaNs
print('Before: ', len(all_features_business))
all_features_business = all_features_business.dropna(axis=0)
print('After:  ', len(all_features_business))

Before:  13943
After:   13922


In [80]:
# First, shuffle the dataframe 
# and reset the index. (Makes for easier handling of train/test later)
all_features_business = all_features_business.sample(frac=1).reset_index(drop=True)

# Create final y and x 
y_df = all_features_business[all_cats]
x_cols = [ele for ele in all_features_business.columns if ele not in all_cats+['categories', 'business_id']]
# May also want to remove from x_cols: 'cool', 'funny', 'useful', 'stars', 'categories', 'review_count' 
# May also want to drop rows that do not contain more than a threshold number of reviews (20?, 100?)
x_df = all_features_business[x_cols]

# Numpy arrays
x = x_df.values
y = y_df.values

# Classifier wants 1/0, not T/F
y = y.astype(int)

# Split into Train/Test sets
def splitSets(x, y, test_size=0.2):
    test_size_absolute = np.int(test_size * len(x))
    X_test, X_train = x[:test_size_absolute,:], x[test_size_absolute:,:]
    y_test, y_train = y[:test_size_absolute,:], y[test_size_absolute:,:]
    return X_train, X_test, y_train, y_test
    
test_size = 0.2
X_train, X_test, y_train, y_test = splitSets(x, y, test_size=test_size)

In [81]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Category Prediction

In [82]:
# Multilabel Classification
# RandomForestClassifier supports multilabel classification

# Most other classifiers will require use of 
    # sklearn.multioutput.MultiOutputClassifier to run a separate classifier model for each targe
    
from sklearn.ensemble import RandomForestClassifier

In [83]:
rfc = RandomForestClassifier(n_estimators=10, n_jobs=-1)

In [84]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Recall (and other classification metrics)

In our case we want a Recall = TPR (True Positive Rate) close to 1 since we want to Recall ALL correct categories. 

The only requirement we have for Precision is that it be less than 1. This is because we want some FPs (False Positives) since these are what WE ARE RECOMMENDING!!

In [85]:
from sklearn.metrics import classification_report

y_predict = rfc.predict(X_test)
print(classification_report(y_test, y_predict, target_names=all_cats))

                                precision    recall  f1-score   support

                     Nightlife       0.84      0.30      0.44       233
                    SportsBars       0.00      0.00      0.00        36
                   Restaurants       0.91      0.88      0.89       964
                          Bars       0.81      0.27      0.41       259
         American(Traditional)       0.00      0.00      0.00         0
                         Pizza       0.97      0.31      0.47       106
                   HairRemoval       0.33      0.02      0.03        66
               NailTechnicians       0.00      0.00      0.00         4
                   Beauty&Spas       0.99      0.58      0.73       264
                    NailSalons       0.92      0.45      0.61        75
                        Waxing       0.50      0.02      0.05        41
                       DaySpas       1.00      0.03      0.05        37
                   Electronics       0.00      0.00      0.00  

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/daviderickson/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/daviderickson/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [86]:
from sklearn.metrics import recall_score 

recall_all_cats = recall_score(y_test, y_predict, average=None)
recall_all_cats

/Users/daviderickson/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


array([0.30042918, 0.        , 0.87966805, ..., 0.        , 0.        ,
       0.        ])

## Top RECOMMENDATIONS

Look at the top NONMATCHING RESULTS, which are the top recommendations!

In [87]:
y_proba = rfc.predict_proba(X_test)

In [88]:
print( len(y_proba), ' L')
print( len(y_proba[0]), ' W')
print( len(y_proba[0][0]), " D (0: False prob'y, 1: True prob'y)")

1090  L
2784  W
2  D (0: False prob'y, 1: True prob'y)


In [89]:
y_proba[0][0]

array([0.7, 0.3])

In [90]:
reccs_binary = (y_test == 0) & (y_predict == 1)
reccs_binary

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [91]:
all_cats_ser = pd.Series(data=all_cats)

In [92]:
all_cats_true = []
all_cats_recc = []
for biz in range(len(y_test)):
    cats_true = ', '.join(list(all_cats_ser[y_test[biz,:]==1]))
    all_cats_true.append(cats_true)
    
    cats_recc = ', '.join(list(all_cats_ser[reccs_binary[biz,:]==True]))
    all_cats_recc.append(cats_recc)

reccs_df = pd.DataFrame(data=all_cats_true, columns=['Labeled'])
reccs_df['Recommended'] = all_cats_recc
reccs_df.tail()

,Labeled,Recommended
2779,"Veterinarians, Pets",
2780,"Food, ConvenienceStores",Shopping
2781,"Shopping, Home&Garden, HomeDecor, Kitchen&Bath",
2782,"Food, IceCream&FrozenYogurt",Restaurants
2783,"LocalServices, PestControl",


In [93]:
list(all_features_business.columns)

['business_id',
 'w2v0',
 'w2v1',
 'w2v2',
 'w2v3',
 'w2v4',
 'w2v5',
 'w2v6',
 'w2v7',
 'w2v8',
 'w2v9',
 'w2v10',
 'w2v11',
 'w2v12',
 'w2v13',
 'w2v14',
 'w2v15',
 'w2v16',
 'w2v17',
 'w2v18',
 'w2v19',
 'w2v20',
 'w2v21',
 'w2v22',
 'w2v23',
 'w2v24',
 'w2v25',
 'w2v26',
 'w2v27',
 'w2v28',
 'w2v29',
 'w2v30',
 'w2v31',
 'w2v32',
 'w2v33',
 'w2v34',
 'w2v35',
 'w2v36',
 'w2v37',
 'w2v38',
 'w2v39',
 'w2v40',
 'w2v41',
 'w2v42',
 'w2v43',
 'w2v44',
 'w2v45',
 'w2v46',
 'w2v47',
 'w2v48',
 'w2v49',
 'w2v50',
 'w2v51',
 'w2v52',
 'w2v53',
 'w2v54',
 'w2v55',
 'w2v56',
 'w2v57',
 'w2v58',
 'w2v59',
 'w2v60',
 'w2v61',
 'w2v62',
 'w2v63',
 'w2v64',
 'w2v65',
 'w2v66',
 'w2v67',
 'w2v68',
 'w2v69',
 'w2v70',
 'w2v71',
 'w2v72',
 'w2v73',
 'w2v74',
 'w2v75',
 'w2v76',
 'w2v77',
 'w2v78',
 'w2v79',
 'w2v80',
 'w2v81',
 'w2v82',
 'w2v83',
 'w2v84',
 'w2v85',
 'w2v86',
 'w2v87',
 'w2v88',
 'w2v89',
 'w2v90',
 'w2v91',
 'w2v92',
 'w2v93',
 'w2v94',
 'w2v95',
 'w2v96',
 'w2v97',
 'w2v98',
 'w2

In [94]:
reccs_df['categories'] = all_features_business['categories'].iloc[:len(reccs_df)]
reccs_df['business_id'] = all_features_business['business_id'].iloc[:len(reccs_df)]
reccs_df.tail()

,Labeled,Recommended,categories,business_id
2779,"Veterinarians, Pets",,"Pets, Veterinarians",XCNFeIUMsMV5_NaHi6bHGw
2780,"Food, ConvenienceStores",Shopping,"Food, Convenience Stores",lHZgNaqcTTTu2qX8j2qzxA
2781,"Shopping, Home&Garden, HomeDecor, Kitchen&Bath",,"Shopping, Home & Garden, Home Decor, Kitchen & Bath",H7rpWv02D6WTu6IpNNDkWw
2782,"Food, IceCream&FrozenYogurt",Restaurants,"Food, Ice Cream & Frozen Yogurt",EwN1LCoJXB0z_a-LxLFKyQ
2783,"LocalServices, PestControl",,"Pest Control, Local Services",Q_BzLkCJWf0iB1wfRWcnZw


In [95]:
# This is where I need to pick up. I need to match the dataframes 
# so that I can match reviews etc and judge how well the recommender is doing

In [96]:
list(all_features_business['categories'].tail())

['Chicken Wings, Fast Food, Restaurants, Chicken Shop',
 'Home Services, Junk Removal & Hauling, Hotels & Travel, Piano Services, Automotive, Musical Instrument Services, Movers, Local Services, Couriers & Delivery Services, Vehicle Shipping, Transportation',
 'Local Services, Dry Cleaning & Laundry, Laundry Services',
 'Food, Street Vendors, Farmers Market',
 'Real Estate, Home Services, Apartments']

In [97]:
len(reccs_df[reccs_df['Recommended']!=''])

296

In [98]:
reccs_df[reccs_df['Recommended']!=''].sort_values(by='Recommended')

,Labeled,Recommended,categories,business_id
998,"Arts&Entertainment, Festivals",ActiveLife,"Festivals, Arts & Entertainment",8PhLT_WRxnrZfKWICllMUQ
643,"Nightlife, Bars, EventPlanning&Services, ActiveLife, Venues&EventSpaces, Pubs, RecreationCenters, SportsClubs, Beaches, BeachBars, BeachVolleyball",Arts&Entertainment,"Recreation Centers, Pubs, Beach Bars, Bars, Nightlife, Active Life, Venues & Event Spaces, Beach Volleyball, Event Planning & Services, Beaches, Sports Clubs",8XSC5y10zpdDITOC-pUK5Q
885,"Restaurants, Vietnamese",AsianFusion,"Vietnamese, Restaurants",KEGLWeFAWXvo0W2LnujhtQ
1526,"Automotive, AutoParts&Supplies",AutoRepair,"Automotive, Auto Parts & Supplies",QgXTuzc_i8dO_p2XD7PP0Q
2088,"Automotive, AutoGlassServices, Wheel&RimRepair, Tires, CarWash",AutoRepair,"Tires, Wheel & Rim Repair, Automotive, Auto Glass Services, Car Wash",qNChnVaoWDS-RFd9NDBPEA
741,"Automotive, CarDealers, AutoParts&Supplies",AutoRepair,"Auto Parts & Supplies, Automotive, Car Dealers",gazN60LwAxWUVk2xo_3KbA
780,"Automotive, BodyShops",AutoRepair,"Automotive, Body Shops",va8PGeWUCaCnKsRPXGp1LQ
1128,"Automotive, Hotels, Hotels&Travel, MotorcycleRepair, MotorcycleRental, MotorcycleDealers",AutoRepair,"Motorcycle Repair, Motorcycle Rental, Motorcycle Dealers, Automotive, Hotels & Travel",-ucQnELMVRIUOi3-Kv5r0Q
978,"Automotive, AutoParts&Supplies",AutoRepair,"Auto Parts & Supplies, Automotive",jubHvev4-PO1PBNHFD73vw
775,"Automotive, Wheel&RimRepair, Tires, AutoParts&Supplies","AutoRepair, OilChangeStations","Wheel & Rim Repair, Automotive, Auto Parts & Supplies, Tires",FlUimPjl-6AkZk0neebkWA


In [99]:
dfreviews.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')

In [100]:
pd.set_option('display.max_colwidth',2000)
dfreviews[dfreviews['business_id']=='QZkSIa1Be9QIFLD6NUBxPQ']['text']

24768                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [101]:
# Given X_test row, find identical row in all_features_business and use that to find 'business_id'
all_features_business[x_cols].values == X_test[0]

array([[ True,  True,  True, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [102]:
X_test.shape

(2784, 305)

In [103]:
all_features_business[x_cols].values.shape

(13922, 305)